In [6]:
import time
import pandas as pd
import dask.dataframe as dd
import numpy as np
import feather
import matplotlib.pyplot as plt
import seaborn as sns
import gc
from imblearn.over_sampling import SMOTE, SMOTENC, SVMSMOTE, ADASYN, BorderlineSMOTE, RandomOverSampler
from sklearn.metrics import *
from imblearn.metrics import classification_report_imbalanced
import datetime
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from collections import Counter
import configparser
import ast
import pickle
import sys
from multiprocessing import Pool, Process
import warnings
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.manifold import TSNE
from sklearn.linear_model import LogisticRegressionCV
from sklearn.cluster import DBSCAN
import hdbscan

config = configparser.ConfigParser()

config_path = config.read('/home/melgazar9/Trading/TD/Scripts/Trading-Scripts/CL/scripts/CL_10min_TRAIN-MODEL.ini')

print('**************** RUNNING', config['NAME']['product'], ' ****************')

train_start_date = config['PARAMS']['train_start_date']
train_end_date = config['PARAMS']['train_end_date']
val_start_date = config['PARAMS']['val_start_date']
val_end_date = config['PARAMS']['val_end_date']
test_start_date = config['PARAMS']['test_start_date']

threshold = float(config['PARAMS']['threshold'])
multiplier = float(config['PARAMS']['multiplier'])

strong_buy_actual = float(config['PARAMS']['strong_buy_actual'])
med_buy_actual = float(config['PARAMS']['med_buy_actual'])
no_trade_actual = float(config['PARAMS']['no_trade_actual'])
med_sell_actual = float(config['PARAMS']['med_sell_actual'])
strong_sell_actual = float(config['PARAMS']['strong_sell_actual'])
stop_actual = float(config['PARAMS']['stop_actual'])

strong_buy_HL = float(config['PARAMS']['strong_buy_HL'])
med_buy_HL = float(config['PARAMS']['med_buy_HL'])
no_trade_HL = float(config['PARAMS']['no_trade_HL'])
med_sell_HL = float(config['PARAMS']['med_sell_HL'])
strong_sell_HL = float(config['PARAMS']['strong_sell_HL'])
stop_HL = float(config['PARAMS']['stop_HL'])

strong_cap_actual = float(config['PARAMS']['strong_cap_actual'])
med_cap_actual = float(config['PARAMS']['med_cap_actual'])
strong_cap_HL = float(config['PARAMS']['strong_cap_HL'])
med_cap_HL = float(config['PARAMS']['med_cap_actual'])

min_prob0 = float(config['PARAMS']['min_prob0'])
min_prob1 = float(config['PARAMS']['min_prob1'])
min_prob3 = float(config['PARAMS']['min_prob3'])
min_prob4 = float(config['PARAMS']['min_prob4'])

min_prob_override0 = float(config['PARAMS']['min_prob_override0'])
min_prob_override1 = float(config['PARAMS']['min_prob_override1'])
min_prob_override3 = float(config['PARAMS']['min_prob_override3'])
min_prob_override4 = float(config['PARAMS']['min_prob_override4'])

Actual_Move = config['PARAMS']['Actual_Move_Name']
Actual_HighMove = config['PARAMS']['Actual_HighMove_Name']
Actual_LowMove = config['PARAMS']['Actual_LowMove_Name']

round_trip_fee = float(config['PARAMS']['round_trip_fee'])

if config['PARAMS']['read_csv'] == 'TRUE':
    print('Reading CSV...')
    ddf = dd.read_csv(config['PATH']['filename'])
elif config['PARAMS']['read_feather'] == 'TRUE':
    print('Reading Feather File...')
    # df = pd.read_feather(config['PATH']['filename'], use_threads=32)
    df = feather.read_dataframe(config['PATH']['filename'])
elif config['PARAMS']['read_parquet'] == 'TRUE':
    ddf = dd.read_parquet(config['PATH']['filename'])

df = ddf.compute()

try:
    df.set_index('Datetime', inplace=True)
except KeyError:
    print('Datetime index is already set!')
    pass


df.index = pd.to_datetime(df.index)
df.index = df.index.tz_localize('utc').tz_convert('US/Central')

df.dropna(axis=0, inplace=True)

df[Actual_Move] = df[['Prev' + Actual_Move.strip('Actual')]].resample(config['NAME']['product'][3:5] + 'min').first().rename(columns={'Prev' + Actual_Move.strip('Actual') : Actual_Move}).shift(-1)
df[Actual_HighMove] = df[['Prev' + Actual_HighMove.strip('Actual')]].resample(config['NAME']['product'][3:5] + 'min').first().rename(columns={'Prev' + Actual_HighMove.strip('Actual') : Actual_HighMove}).shift(-1)
df[Actual_LowMove] = df[['Prev' + Actual_LowMove.strip('Actual')]].resample(config['NAME']['product'][3:5] + 'min').first().rename(columns={'Prev' + Actual_LowMove.strip('Actual'): Actual_LowMove}).shift(-1)

df.head()

/home/melgazar9/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/home/melgazar9/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/home/melgazar9/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/melgazar9/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/home/melgazar9/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/home/melgazar9/anacon

**************** RUNNING CL_10min_  ****************
Datetime index is already set!


,Prev1minMove,Prev1minRange,Prev1minHighMove,Prev1minLowMove,Prev1minVolume,Prev2minMove,Prev2minRange,Prev2minHighMove,Prev2minLowMove,Prev2minVolume,...,Prev5minLowMove_Opinion,Prev10minMove_Opinion,Prev10minHighMove_Opinion,Prev10minLowMove_Opinion,Prev15minMove_Opinion,Prev15minHighMove_Opinion,Prev15minLowMove_Opinion,Actual10minMove,Actual10minHighMove,Actual10minLowMove
Datetime,,,,,,,,,,,,,,,,,,,,,
2015-01-01 18:00:00-06:00,-0.05,0.05,0.00,-0.05,28.0,-0.04,0.08,0.04,-0.04,69.0,...,0,1,1,0,1,1,0,0.03,0.18,-0.17
2015-01-01 18:01:00-06:00,-0.07,0.12,0.02,-0.10,102.0,-0.04,0.08,0.04,-0.04,69.0,...,0,1,1,0,1,1,0,NaN,NaN,NaN
2015-01-01 18:02:00-06:00,0.00,0.11,0.00,-0.11,111.0,-0.06,0.19,0.02,-0.17,213.0,...,0,1,1,0,1,1,0,NaN,NaN,NaN
2015-01-01 18:03:00-06:00,0.23,0.29,0.29,0.00,156.0,-0.06,0.19,0.02,-0.17,213.0,...,0,1,1,0,1,1,0,NaN,NaN,NaN
2015-01-01 18:04:00-06:00,-0.04,0.11,0.02,-0.09,75.0,0.22,0.29,0.29,0.00,231.0,...,0,1,1,0,1,1,0,NaN,NaN,NaN


In [10]:
X = df.drop([i for i in df.columns if i=='Target_HL' or 'Actual' in i], axis=1)
# y = df['Target_HL']

In [11]:
X.ffill(inplace=True)
# y.ffill(inplace=True)
# y.fillna(2, inplace=True)

In [12]:
warnings.filterwarnings('ignore')
scaler = StandardScaler().fit(X)
X_scaled = pd.DataFrame(scaler.transform(X), columns=X.columns)

X_scaled.head()

,Prev1minMove,Prev1minRange,Prev1minHighMove,Prev1minLowMove,Prev1minVolume,Prev2minMove,Prev2minRange,Prev2minHighMove,Prev2minLowMove,Prev2minVolume,...,PrevDaily_VWAP_1minAgg,Prev5minMove_Opinion,Prev5minHighMove_Opinion,Prev5minLowMove_Opinion,Prev10minMove_Opinion,Prev10minHighMove_Opinion,Prev10minLowMove_Opinion,Prev15minMove_Opinion,Prev15minHighMove_Opinion,Prev15minLowMove_Opinion
0,-1.458618,0.201137,-0.653221,-0.796371,-0.493710,-0.873845,0.387687,0.303838,-0.229491,-0.518203,...,0.263368,-0.077752,-0.122836,-0.138993,8.306733,5.296185,-0.202676,6.373594,4.119662,-0.255192
1,-2.043249,1.739075,0.012754,-2.180312,-0.386211,-0.873845,0.387687,0.303838,-0.229491,-0.518203,...,0.259422,-0.077752,-0.122836,-0.138993,8.306733,5.296185,-0.202676,6.373594,4.119662,-0.255192
2,0.002960,1.519369,-0.653221,-2.457100,-0.373137,-1.312122,2.313779,-0.204121,-3.160738,-0.405666,...,0.259386,-0.077752,-0.122836,-0.138993,8.306733,5.296185,-0.202676,6.373594,4.119662,-0.255192
3,6.726219,5.474066,9.003407,0.587571,-0.307766,-1.312122,2.313779,-0.204121,-3.160738,-0.405666,...,0.266485,-0.077752,-0.122836,-0.138993,8.306733,5.296185,-0.202676,6.373594,4.119662,-0.255192
4,-1.166302,1.519369,0.012754,-1.903524,-0.425434,4.823754,4.064771,6.653331,0.672431,-0.391598,...,0.268068,-0.077752,-0.122836,-0.138993,8.306733,5.296185,-0.202676,6.373594,4.119662,-0.255192


In [15]:
dbs = DBSCAN(n_jobs=32).fit(X_scaled)
dbs

DBSCAN(algorithm='auto', eps=0.5, leaf_size=30, metric='euclidean',
    metric_params=None, min_samples=5, n_jobs=32, p=None)

In [17]:
dbs.labels_

array([-1, -1, -1, ..., -1, -1, -1])

In [20]:
from sklearn import metrics

core_samples_mask = np.zeros_like(dbs.labels_, dtype=bool)
core_samples_mask[dbs.core_sample_indices_] = True
labels = dbs.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)
# print("Homogeneity: %0.3f" % metrics.homogeneity_score(labels_true, labels))
# print("Completeness: %0.3f" % metrics.completeness_score(labels_true, labels))
# print("V-measure: %0.3f" % metrics.v_measure_score(labels_true, labels))
# print("Adjusted Rand Index: %0.3f" % metrics.adjusted_rand_score(labels_true, labels))
# print("Adjusted Mutual Information: %0.3f" % metrics.adjusted_mutual_info_score(labels_true, labels))
print("Silhouette Coefficient: %0.3f" % metrics.silhouette_score(X_scaled, labels))

# Plot result

# Black removed and is used for noise instead.

unique_labels = set(labels)
colors = [plt.cm.Spectral(each)
          for each in np.linspace(0, 1, len(unique_labels))]
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]

    class_member_mask = (labels == k)

    xy = X_scaled[class_member_mask & core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=14)

    xy = X_scaled[class_member_mask & ~core_samples_mask]
    plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=6)

plt.title('Estimated number of clusters: %d' % n_clusters_)
plt.show()

Estimated number of clusters: 0
Estimated number of noise points: 1193531


ValueError: Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)

In [22]:
pd.Series(dbs.labels_).value_counts()

-1    1193531
dtype: int64